# Projectile Motion with Linear Air Resistance: Euler's Method

## 1. Physical Scenario
We are simulating the trajectory of a particle launched from the origin $(0,0)$ with:
* **Initial Velocity ($v_0$):** $100$ m/s
* **Launch Angles ($\theta$):** Three chosen values (e.g., $30^\circ, 45^\circ, 60^\circ$)
* **Gravity ($g$):** $10$ m/s $^2$
* **Linear Drag Coefficient ($k = B/m$):** We will use a value that makes the drag visible (e.g., $0.1$ s $^{-1}$) to observe the effect clearly.

## 2. The Objective
To compare "Ideal" (vacuum) trajectories with "Linear Drag" trajectories. Linear drag (Stokes' drag) is often used for objects moving through viscous fluids or at lower speeds.

## 2. Mathematical Equations (Linear Model)

### Without Air Resistance ($B = 0$)
The acceleration is constant:
$$a_x = 0, \quad a_y = -g$$

### With Linear Air Resistance
The drag force is $\vec{F}_d = -B\vec{v}$. Dividing by mass ($m$), we get the acceleration components:
$$\frac{dv_x}{dt} = -k v_x$$
$$\frac{dv_y}{dt} = -g - k v_y$$
where $k = B/m$ is the drag per unit mass.

### Euler's Method Update
Using a small time step $\Delta t$:
1. $v_{x, i+1} = v_{x, i} + (-k v_{x, i}) \cdot \Delta t$
2. $v_{y, i+1} = v_{y, i} + (-g - k v_{y, i}) \cdot \Delta t$
3. $x_{i+1} = x_i + v_{x, i} \cdot \Delta t$
4. $y_{i+1} = y_i + v_{y, i} \cdot \Delta t$